# Read and write from Spark to SQL using the MSSQL Spark Connector
A typical big data scenario a key usage pattern is high volume, velocity and variety data processing in Spark followed with batch or streaming writes to SQL for access to LOB applications. These usage patterns greatly benefit from a connector that utilizes key SQL optimizations and provides an efficient write to SQLServer master instance and SQL Server data pool in Big Data Clusters.

MSSQL Spark connector provides an efficient write SQLServer master instance and SQL Server data pool in Big Data Clusters.

Usage
----
- Familiar Spark DataSource V1 interface
- Referenced by the name "mssql" or fully qualified "com.microsoft.sqlserver.jdbc.spark"
- Use from supported Spark language bindings ( Python, Scala, Java, R)
- Optionally pass Bulk Copy parameters 


<img src =
"C:\repos\sql-server-samples\samples\features\sql-big-data-cluster\spark\spark_to_sql\MSSQL_Spark_Connector.jpg" style="float: center;" alt="drawing" width="900">

More details
-----------

MSSQL Spark connector, uses [SQL Server Bulk copy APIS](https://docs.microsoft.com/en-us/sql/connect/jdbc/using-bulk-copy-with-the-jdbc-driver?view=sql-server-2017#sqlserverbulkcopyoptions) to implement an efficient write to SQL Server. The connector is based on Spark Data source APIs and provides a familiar JDBC interface for access

The Sample
---------

The following sample shows MSSQL JDBC Connector for read/write QLServer master instance and SQL Server data pool in Big Data Clusters. The sample is divided into 2 parts. The first part shows read/write to SQL Master instance and Part 2 shows read/write to Data Pools in Big Data Cluster. 

In the sample we' ll 
- Read a file from HDFS and do some basic processing 
- In Part 1, we'll write the dataframe to SQL server table and then read the table to a dataframe .
- In Part 2, we'll write the dataframe to SQL Server data pool external table and then read it back to a spark data frame. 

PreReq: 
- The sample uses a SQL database named "MyTestDatabase". Create this before you run this sample. The database can be created as follows
    ``` sql
    Create DATABASE MyTestDatabase
    GO 
    ``` 
- Download [AdultCensusIncome.csv]( https://amldockerdatasets.azureedge.net/AdultCensusIncome.csv ) to your local machine.  Create a hdfs folder named spark_data and upload the file there. 
- [For CTP2.5] Configure the spark session to use the MSSQL Connector jar. The jar can be found at /jar/spark-mssql-connector-assembly-1.0.0.jar post deployment of Big Data Cluster.

``` sh
    %%configure -f
    {"conf": {"spark.jars": "/jar/spark-mssql-connector-assembly-1.0.0.jar"}}
```


    

# Configure the notebook to use the MSSQL Spark connector
This step is only required for CTP2.5 to point the spark session to the relevant jar.
 

In [3]:
%%configure -f
{"conf": {"spark.jars": "/jar/spark-mssql-connector-assembly-1.0.0.jar"}}


# Read CSV into a data frame
In this step we read the CSV into a data frame and do some basic cleanup steps. 




In [4]:
#spark = SparkSession.builder.getOrCreate()
sc.setLogLevel("WARN")

#Read a file and then write it to the SQL table
datafile = "/spark_data/AdultCensusIncome.csv"
df = spark.read.format('csv').options(header='true', inferSchema='true', ignoreLeadingWhiteSpace='true', ignoreTrailingWhiteSpace='true').load(datafile)
df.show(5)


#Process this data. Very simple data cleanup steps. Replacing "-" with "_" in column names
columns_new = [col.replace("-", "_") for col in df.columns]
df = df.toDF(*columns_new)
df.show(5)


+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|education|education-num|    marital-status|       occupation| relationship| race|   sex|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 39|       State-gov| 77516|Bachelors|           13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States| <=50K|
| 50|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States| <=50K|
| 38|         Private|215646|  HS-grad|            9|          Divorced|Handlers-cleaners|Not-i

# (PART 1) Write and READ to SQL Table
- Write dataframe to SQL table to SQL Master
- Read SQL Table to Spark dataframe

In [5]:
#Write from Spark to SQL table using MSSQL Spark Connector
print("Use MSSQL connector to write to master SQL instance ")

servername = "jdbc:sqlserver://master-0.master-svc"
dbname = "MyTestDatabase"
url = servername + ";" + "databaseName=" + dbname + ";"

dbtable = "AdultCensus"
user = "**"
password = "**" # Please specify password here

#com.microsoft.sqlserver.jdbc.spark

try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", dbtable) \
    .option("user", user) \
    .option("password", password) \
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(overwrite) succeeded  ")




Use MSSQL connector to write to master SQL instance 
MSSQL Connector write succeeded

In [6]:
#Use mode as append
try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("append") \
    .option("url", url) \
    .option("dbtable", dbtable) \
    .option("user", user) \
    .option("password", password) \
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(append) succeeded  ")

In [7]:
#Read from SQL table using MSSQ Connector
print("read data from SQL server table  ")
jdbcDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", dbtable) \
        .option("user", user) \
        .option("password", password).load()

jdbcDF.show(5)

read data from SQL server table  
+---+---------+------+------------+-------------+------------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|   education|education_num|    marital_status|       occupation|  relationship| race|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+---------+------+------------+-------------+------------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+------+
| 22|  Private|151105|Some-college|           10|     Never-married|            Sales|Other-relative|White|Female|           0|           0|            18| United-States| <=50K|
| 38|  Private|179117|  Assoc-acdm|           12|     Never-married|Machine-op-inspct| Not-in-family|Black|Female|       10520|           0|            50| United-States|  >50K|
| 72|        ?| 33608|Some-college|           10|Married-civ-spouse|        

# (PART 2) Write and READ to Data Pool external Tables in Big Data Cluster
- Write dataframe to SQL external table in Data Pools in Big Data Cluste
- Read SQL external Table to Spark dataframe

In [8]:
#Write from Spark to SQL table using MSSQL Spark Connector
print("Use MSSQL connector to write to master SQL instance ")

datapool_table = "AdultCensus_DataPoolTable"
datasource_name = "test_data_src"


try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", datapool_table) \
    .option("user", user) \
    .option("password", password) \
    .option("dataPoolDataSource",datasource_name)\
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(overwrite) to data pool external table succeeded")


Use MSSQL connector to write to master SQL instance 
MSSQL Connector write to data pool external table succeeded

In [9]:
try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("append") \
    .option("url", url) \
    .option("dbtable", datapool_table) \
    .option("user", user) \
    .option("password", password) \
    .option("dataPoolDataSource",datasource_name)\
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(append) to data pool external table succeeded")

MSSQL Connector write(append) to data pool external table succeeded

In [7]:
#Read from SQL table using MSSQ Connector
print("read data from SQL server table  ")
jdbcDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", datapool_table) \
        .option("user", user) \
        .option("password", password)\
        .load()

jdbcDF.show(5)

print("MSSQL Connector read from data pool external table succeeded")

An error occurred while calling o152.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 4 times, most recent failure: Lost task 0.3 in stage 8.0 (TID 15, storage-0-1.storage-0-svc.newaks.svc.cluster.local, executor 1): com.microsoft.sqlserver.jdbc.SQLServerException: 105080;CREATE EXTERNAL TABLE failed because the URI contains an unsupported path, fragment, user info, or query. Revise the URI and try again.
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDatabaseError(SQLServerException.java:256)
	at com.microsoft.sqlserver.jdbc.SQLServerResultSet$FetchBuffer.nextRow(SQLServerResultSet.java:5427)
	at com.microsoft.sqlserver.jdbc.SQLServerResultSet.fetchBufferNext(SQLServerResultSet.java:1758)
	at com.microsoft.sqlserver.jdbc.SQLServerResultSet.next(SQLServerResultSet.java:1016)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anon$1.getNext(JdbcUtils.scala:352)
	at org.apache.spark.sql.execution.datasour